## Bot Telegram

In [26]:
#pip install pyTelegramBotAPI


In [27]:
pip install pyTelegramBotAPI


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importación de Paquetes

In [28]:
import telebot
from dotenv import load_dotenv
import os
import requests
from datetime import datetime

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

In [30]:
from telebot.types import InlineKeyboardMarkup
from telebot.types import InlineKeyboardButton

In [31]:
CSV_FILE = "./archivos/doc/operaciones.csv"

if os.path.exists(CSV_FILE):
    df = pd.read_csv(CSV_FILE)
    df["user_id"] = df["user_id"].astype(int)
    df["monto"] = df["monto"].astype(float)
else:
    df = pd.DataFrame(columns=["user_id", "tipo", "monto", "fecha"])

In [32]:
### Instanciamos el bot de telegram
### el único parámetro obligatorio para instanciarlo es el TOKEN.

In [33]:
# importamos nuestro token
load_dotenv()
TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')
bot_databanda = telebot.TeleBot(TELEGRAM_TOKEN)

## Responder a menasjes que son comandos (básicos)

In [34]:
# RESPONDER A MENSAJES QUE SON COMANDOS

@bot_databanda.message_handler(commands=['start'])
def cmd_start(message):
    bot_databanda.reply_to(message, "Hola, soy un bot de finanzas. ¿Cómo puedo ayudarte?")

@bot_databanda.message_handler(commands=['fin'])
def cmd_fin(message):
    bot_databanda.reply_to(message, "Nos vemos, recuerda que puedes consultar tu saldo o realizar transferencias.")

@bot_databanda.message_handler(commands=['help'])
def send_help(message):
    bot_databanda.reply_to(message, "Aquí tienes una lista de comandos que puedes usar:\n"
                                     "/start - Iniciar el bot\n"
                                     "/help - Mostrar esta ayuda\n"
                                     "/balance - Consultar tu saldo\n"
                                     "/ingresar - Ingresar monto\n"
                                     "/gasto - Registrar un gasto\n"
                                     "/cotizaciones - Consultar cotizaciones\n"
                                     "/criptomonedas - Consultar si puedo comprar\n"
                                     "/historial - Consultar historial\n"
                                     "/fin - Finalizar la conversación")
                                     

## Funciones más complejas (heavys)

In [35]:
#que el usuario pueda consultar su saldo si tiene 0 que pueda agregar en base a transferencia

# Diccionario para guardar el saldo de cada usuario
user_saldos = {}
from telebot import types  # Asegúrate de tener esto arriba


def get_price(moneda, cambio):
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        "ids": moneda.lower(),
        "vs_currencies": cambio.lower()
    }
    r = requests.get(url, params=params)
    data = r.json()
    return data.get(moneda.lower(), {}).get(cambio.lower())

def calcular_saldo(user_id):
    user = df[df["user_id"] == int(user_id)]
    if user.empty:
        return 0.0
    ingresos = user[user["tipo"] == "ingreso"]["monto"].sum()
    gastos = user[user["tipo"] == "gasto"]["monto"].sum()
    return ingresos - gastos

def registrar_operacion(user_id, tipo, monto):
    global df
    nueva_op = pd.DataFrame([{
        "user_id": user_id,
        "tipo": tipo,
        "monto": monto,
        "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }])
    df = pd.concat([df, nueva_op], ignore_index=True)
    df.tail(1).to_csv(CSV_FILE, mode="a", header=not os.path.exists(CSV_FILE), index=False)

@bot_databanda.message_handler(commands=['gasto'])
def cmd_gasto(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_gasto)

def process_gasto(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_gasto)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el gasto?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_gasto)

def confirm_gasto(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        # Sumar el monto al saldo del usuario
        registrar_operacion(user_id, "gasto", amount)
        bot_databanda.send_message(message.chat.id, f"✅ Gasto de ${amount} registrado con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Gasto cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el gasto?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

@bot_databanda.message_handler(commands=['ingresar'])
def cmd_ingresar(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_amount)

def process_amount(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_amount)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_amount)

def confirm_ingreso(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        registrar_operacion(user_id, "ingreso", amount)  #Guardar en pandas
        bot_databanda.send_message(message.chat.id, f"✅ Ingreso de ${amount} realizada con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Ingreso cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

@bot_databanda.message_handler(commands=['balance'])
def send_balance(message):
    user_id = message.from_user.id
    saldo = calcular_saldo(user_id)
    bot_databanda.reply_to(message, f"Tu saldo es de ${saldo}")

@bot_databanda.message_handler(commands=['historial'])
def send_historial(message):
    user_id = message.from_user.id
    user=df[df["user_id"] == user_id].tail(5)
    if user.empty:
        bot_databanda.reply_to(message, "📂 No tienes operaciones registradas todavía.")
        return
    historial_texto = "📂 Tus últimos 5 movimientos:\n\n"
    for _, row in user.iterrows():
        historial_texto += f"{row['fecha']} - {row['tipo'].capitalize()}: ${row['monto']}\n"
    
    bot_databanda.reply_to(message, historial_texto)

@bot_databanda.message_handler(commands=['criptomonedas'])
def criptomonedas(message):
    user_id = message.from_user.id
    saldo = calcular_saldo(user_id)  # saldo en ARS, según tu lógica
    if saldo <= 0:
        bot_databanda.reply_to(message, "⚠️ No tienes saldo disponible para comprar.")
        return

    monedas = ["bitcoin", "ethereum", "litecoin", "dogecoin", "tether"]
    resultados = []

    for moneda in monedas:
        precio = get_price(moneda, "ars")
        if precio:
            cantidad = saldo / precio
            resultados.append(f"   - {cantidad:.6f} {moneda.upper()}")

    texto = f"💰 Con tu saldo de ${saldo:.2f} ARS puedes comprar:\n" + "\n".join(resultados)
    bot_databanda.reply_to(message, texto)

In [36]:
# RESPONDER A MENSAJES QUE SON TEXTO
#@bot_databanda.message_handler(content_types=['text'])

def bot_mensajes_testo(message):
    bot_databanda.reply_to(message.chat.id, "Me enviaste un mensaje de texto: " + message)


In [ ]:
bot_databanda.set_my_commands([
    telebot.types.BotCommand("/start", "Iniciar el bot"),
    telebot.types.BotCommand("/help", "Mostrar ayuda"),
    telebot.types.BotCommand("/balance", "Consultar tu saldo"),
    telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
    telebot.types.BotCommand("/gasto", "Registrar un gasto"),
    telebot.types.BotCommand("/cotizaciones", "Consultar cotizaciones"),
    telebot.types.BotCommand("/historial", "Consultar historial"),
    telebot.types.BotCommand("/criptomonedas", "Consultar si puedo comprar"),
    telebot.types.BotCommand("/fin", "Finalizar la conversación")
])

True

## OBTENER COTIZACIÓN DÓLAR

In [38]:
# Carpeta para guardar imágenes
if not os.path.exists("archivos/img"):
    os.makedirs("archivos/img")

# Monedas que pueden mostrar gráfica (solo USD)
monedas_con_grafico = ["oficial", "blue"]

# Comando /cotizaciones
@bot_databanda.message_handler(commands=['cotizaciones'])
def cmd_cotizaciones(message):
    markup = InlineKeyboardMarkup()
    botones = [
        InlineKeyboardButton("💵 Dólar Oficial", callback_data="oficial"),
        InlineKeyboardButton("🔵 Dólar Blue", callback_data="blue"),
        InlineKeyboardButton("📈 Dólar MEP", callback_data="bolsa"),
        InlineKeyboardButton("💶 Euro", callback_data="eur"),
        InlineKeyboardButton("🇧🇷 Real", callback_data="brl")
    ]
    for b in botones:
        markup.add(b)
    bot_databanda.send_message(message.chat.id, "Selecciona la cotización que quieras ver:", reply_markup=markup)

# Obtener cotización actual
def get_cotizacion(tipo):
    try:
        # USD: oficial, blue, mep
        if tipo in ["oficial", "blue", "bolsa"]:
            url = "https://dolarapi.com/v1/dolares"
            r = requests.get(url)
            r.raise_for_status()
            data = r.json()
            casa = next((item for item in data if item['casa'].lower() == tipo.lower()), None)
            if casa:
                return f"{casa['nombre']} ({casa['moneda']}):\nCompra: ${casa['compra']} - Venta: ${casa['venta']}\nÚltima actualización: {casa['fechaActualizacion']}"
            else:
                return f"❌ En estos momentos hay mucha demanda para consultar la cotización de: '{tipo}'. Aguarde unos minutos, cualquier cosa será redirigido con nuestro"

        # EUR
        elif tipo == "eur":
            url = "https://dolarapi.com/v1/cotizaciones/eur"
            r = requests.get(url)
            r.raise_for_status()
            item = r.json()
            return f"{item['nombre']} ({item['moneda']}):\nCompra: ${item['compra']} - Venta: ${item['venta']}\nÚltima actualización: {item['fechaActualizacion']}"

        # BRL
        elif tipo == "brl":
            url = "https://dolarapi.com/v1/cotizaciones/brl"
            r = requests.get(url)
            r.raise_for_status()
            item = r.json()
            return f"{item['nombre']} ({item['moneda']}):\nCompra: ${item['compra']} - Venta: ${item['venta']}\nÚltima actualización: {item['fechaActualizacion']}"

        else:
            return f"❌ Moneda '{tipo}' no soportada."

    except Exception as e:
        return f"❌ Ocurrió un error: {e}"



# Manejar selección de moneda
@bot_databanda.callback_query_handler(func=lambda call: True)
def callback_cotizacion(call):
    casa = call.data
    bot_databanda.send_chat_action(call.message.chat.id, "typing")
    cotizacion = get_cotizacion(casa)
    bot_databanda.edit_message_text(chat_id=call.message.chat.id,
                                    message_id=call.message.message_id,
                                    text=cotizacion)

    # Solo USD oficial y blue: preguntar si quiere ver gráfica
    if casa in monedas_con_grafico:
        msg = bot_databanda.send_message(call.message.chat.id,
                                         "¿Deseas ver la evolución histórica de esta moneda en un gráfico? (sí/no)")
        bot_databanda.register_next_step_handler(msg, generar_grafico, casa)


# Generar gráfica solo para USD oficial y blue
def generar_grafico(message, casa):
    respuesta = message.text.strip().lower()
    if respuesta in ["sí", "si"]:
        url = f"https://api.argentinadatos.com/v1/cotizaciones/dolares/{casa}"
        r = requests.get(url)
        if r.status_code == 200:
            data = r.json()
            df = pd.DataFrame(data)
            df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

            # Graficar
            plt.figure(figsize=(10,5))
            plt.plot(df["fecha"], df["venta"], label="Venta", color="blue")
            plt.plot(df["fecha"], df["compra"], label="Compra", color="green", linestyle="--")
            plt.title(f"Evolución del dólar {casa.capitalize()} en Argentina")
            plt.xlabel("Fecha")
            plt.ylabel("Precio (ARS)")
            plt.legend()
            plt.grid(True, linestyle="--", alpha=0.6)
            plt.tight_layout()

            # Guardar imagen
            img_path = f"archivos/img/{casa}_{message.chat.id}.png"
            plt.savefig(img_path)
            plt.close()

            # Enviar imagen al chat
            with open(img_path, "rb") as f:
                bot_databanda.send_photo(message.chat.id, f)
        else:
            bot_databanda.send_message(message.chat.id, "❌ Error al obtener los datos históricos para la gráfica.")
    else:
        bot_databanda.send_message(message.chat.id, "👍 Ok, no se mostrará la gráfica.")



## GRÁFICA DEL USD

## Lanzar el programa /bot

In [ ]:
if __name__ == "__main__":
    print("Iniciando el bot...")

    # Eliminar cualquier webhook activo (previene error 409)
    try:
        bot_databanda.remove_webhook()
    except Exception as e:
        print("No había webhook activo:", e)

    # Seteamos los comandos de nuevo (IMPORTANTE: sin "/")
    bot_databanda.set_my_commands([
        telebot.types.BotCommand("/start", "Iniciar el bot"),
        telebot.types.BotCommand("/help", "Mostrar ayuda"),
        telebot.types.BotCommand("/balance", "Consultar tu saldo"),
        telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
        telebot.types.BotCommand("/gasto", "Registrar un gasto"),
        telebot.types.BotCommand("/cotizaciones", "Consultar cotizaciones"),
        telebot.types.BotCommand("/historial", "Consultar historial"),
        telebot.types.BotCommand("/criptomonedas", "Consultar si puedo comprar"),
        telebot.types.BotCommand("/fin", "Finalizar la conversación")
    ])

    # Arrancar el bot en modo polling
    bot_databanda.infinity_polling()

    print("Fin = Bot Detenido")



    

Iniciando el bot...


2025-09-03 12:24:40,977 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2025-09-03 12:24:40,980 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/telebot/__init__.py", line 1234, in __threaded_polling
    polling_thread.raise_exceptions()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/telebot/util.py", line 112, in raise_exceptions
    raise self.exception_info
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/telebot/util.py", line 94, in run
    task(*args, **kwargs)
    ~~~~^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.f